# ReAct tool Agent

In [7]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


In [8]:
load_dotenv()


True

# Define the state (provide input to agent)
Message is datatype

In [9]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

# Create a tools
# the comments below for each tool like instruct to tools what needd to be done , LLM model understand this and call the related tool do perform the actions

In [10]:
@tool
def add(a: int, b:int):
    """This is an addition function that adds 2 numbers together"""
    return a + b 

@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b

# Add tools

In [11]:
tools = [add, subtract, multiply]

# Bind the tools into chat models 

In [12]:
model = ChatOpenAI(model = "gpt-4o").bind_tools(tools)

# Create the node

In [15]:
def model_call(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content=
        "You are my AI assistant, please answer my query to the best of your ability."
    )
    response = model.invoke([system_prompt] + state["messages"]) #  state["messages" -> is input from user
    return {"messages": [response]}

In [16]:
def should_continue(state: AgentState): 
    messages = state["messages"]
    last_message = messages[-1]
    if not last_message.tool_calls: 
        return "end"
    else:
        return "continue"

# Add node to stateGraph

In [17]:
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)

# Add tool nodes to stateGraph

In [18]:
tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

# Set Entry & Exit point

In [19]:
graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "continue": "tools", # tools node call
        "end": END,
    },
)

graph.add_edge("tools", "our_agent")

In [20]:
app = graph.compile()

In [21]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [22]:
inputs = {"messages": [("user", "Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))


================================ Human Message =================================

Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.
================================== Ai Message ==================================
Tool Calls:
  add (call_N0okHwrTYsc6Doe3vLgnmIcQ)
 Call ID: call_N0okHwrTYsc6Doe3vLgnmIcQ
  Args:
    a: 40
    b: 12
  multiply (call_siGD3MyuV5zA8gWaWzJnOVHe)
 Call ID: call_siGD3MyuV5zA8gWaWzJnOVHe
  Args:
    a: 52
    b: 6
================================= Tool Message =================================
Name: multiply

312
================================== Ai Message ==================================

The result of adding 40 and 12 is 52, and multiplying that result by 6 gives 312.

And here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!
